# Transfer learning tests
Eamon Whalen

In [1]:
import sys
import os
import glob
import numpy as np
import pandas as pd
import altair as alt

from gcnSurrogate.models.feastnetSurrogateModel import FeaStNet
from gcnSurrogate.models.pointRegressorSurrogateModel import PointRegressor
from gcnSurrogate.readers.loadConmechGraphs import loadConmechGraphs
from gcnSurrogate.visualization.altTrussViz import plotTruss, interactiveErrorPlot
from gcnSurrogate.util.gcnSurrogateUtil import *

IndentationError: expected an indented block (altTrussViz.py, line 45)

## 1. Load test data

In [ ]:
dataDir = "data/2D_Truss_v1.3/conmech/"
testDir = os.path.join(dataDir, 'design_9_N_1000/')
allGraphsUnfiltered = loadConmechGraphs(testDir)

# dataDir = "data/endLoadsv1.0/conmech/"
# testDir = os.path.join(dataDir, 'design_7_N_1000/')
# allGraphsUnfiltered = loadConmechGraphs(testDir)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

In [ ]:
testData = filterbyDisp(allGraphsUnfiltered, 0.9)
maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in testData]
source = pd.DataFrame(maxes, columns=['maxes'])
maxDispCutoff = source.max().item()
source.describe()

## 2. Load pre-train data

In [ ]:
# pretrainFiles = glob.glob(os.path.join(dataDir, '*1000.csv'))
# pretrainFiles.remove(testFile)

# allPretrainGraphs = []
# for pretrainFile in pretrainFiles:
#     designName = pretrainDir.split('/')[-1].split('_N')[0]
#     print(f'loading {designName}')
#     graphsUnfiltered = loadGhGraphs(pretrainFile, NUM_DV=5)
#     graphs = filterbyDisp(graphsUnfiltered, 0.9)
#     allPretrainGraphs.extend(graphs)

# print(f'loaded {len(allPretrainGraphs)} pretraining graphs')
# pretrainData, pretrainValData, _ = partitionGraphList(allPretrainGraphs, testSize=0.0, valSize=0.15)

In [ ]:
# pretrainDirs = glob.glob('data/2D_Truss_v1.3/conmech/design_7_N_1000*/')

pretrainDirs = glob.glob(os.path.join(dataDir, '*1000/'))
pretrainDirs.remove(testDir)

allPretrainGraphs = []
for pretrainDir in pretrainDirs:
    designName = pretrainDir.split('/')[-2].split('_N')[0]
    print(f'loading from {pretrainDir}')
    graphsUnfiltered = loadConmechGraphs(pretrainDir)
    graphs = filterbyDisp(graphsUnfiltered, 0.9)
    allPretrainGraphs.extend(graphs)

print(f'loaded {len(allPretrainGraphs)} pretraining graphs')
pretrainData, pretrainValData, _ = partitionGraphList(allPretrainGraphs, testSize=0.0, valSize=0.15)

## 3. Initial training

In [ ]:
saveDir = 'results/transferLrn_endloads_des9_02/'
epochs = 100
ptrGcn = FeaStNet()
history = ptrGcn.trainModel(pretrainData, pretrainValData, 
                            epochs=epochs,
                            saveDir=saveDir+f'preTrain/gcn/')

ptrGcnCheckptFile = ptrGcn.checkptFile
plotHistory(history)

In [ ]:
trainRes = ptrGcn.testModel(pretrainData)
testRes = ptrGcn.testModel(testData) # unseen topology
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

## 4. Transfer learning study

In [ ]:
trainDataDirs = glob.glob(os.path.join(dataDir, 'design_9*/'))
trainDataDirs.remove(testDir)

allResults = []
for trainDataDir in trainDataDirs:
    trainDataUnfiltered = loadConmechGraphs(trainDataDir)
    trainData = filterbyDispValue(trainDataUnfiltered, maxDispCutoff)
    trainSize = len(trainData)
    print(f'loaded train set of size {trainSize}')
    
    
    ### fresh neural network ###
    gcn = FeaStNet()
    history = gcn.trainModel(trainData, trainData, 
                         epochs=epochs, 
                         saveDir=saveDir+f'{trainSize:05}/gcn/')
    
    trainRes = gcn.testModel(trainData)
    trainRes['Model'] = 'Fresh'
    trainRes['Set'] = 'Train'
    trainRes['Train Size'] = trainSize
    allResults.append(trainRes)
    
    testRes = gcn.testModel(testData)
    testRes['Model'] = 'Fresh'
    testRes['Set'] = 'Test'
    testRes['Train Size'] = trainSize
    allResults.append(testRes)
    pd.DataFrame(allResults).to_csv(saveDir+'testResults.csv', index=False)

    
    ### transfer learning ###
    ptrGcn = FeaStNet()
    history = gcn.trainModel(trainData, trainData, 
                             restartFile=ptrGcnCheckptFile,
                             epochs=epochs, 
                             saveDir=saveDir+f'{trainSize:05}/ptrGcn/')
    
    trainRes = gcn.testModel(trainData)
    trainRes['Model'] = 'Transfer learning'
    trainRes['Set'] = 'Train'
    trainRes['Train Size'] = trainSize
    allResults.append(trainRes)
    
    testRes = gcn.testModel(testData)
    testRes['Model'] = 'Transfer learning'
    testRes['Set'] = 'Test'
    testRes['Train Size'] = trainSize
    allResults.append(testRes)
    pd.DataFrame(allResults).to_csv(saveDir+'testResults.csv', index=False)
    
    
    ### random forest ###
    rf = PointRegressor('Random Forest')
    rf.trainModel(trainData, trainData, 
                  useXFeatures=False,
                  saveDir=saveDir+f'{trainSize:05}/rf/')

    trainRes = rf.testModel(trainData)
    trainRes['Model'] = 'Random Forest'
    trainRes['Set'] = 'Train'
    trainRes['Train Size'] = trainSize
    allResults.append(trainRes)
    
    testRes = rf.testModel(testData)
    testRes['Model'] = 'Random Forest'
    testRes['Set'] = 'Test'
    testRes['Train Size'] = trainSize
    allResults.append(testRes)
    pd.DataFrame(allResults).to_csv(saveDir+'testResults.csv', index=False)
    
    
pd.DataFrame(allResults)

In [ ]:
# df = pd.DataFrame(allResults)
df = pd.read_csv('results/transferLrn_endloads_des9_02/testResults.csv')
df

In [ ]:
df = df.replace('Fresh', 'GCN')
df = df.replace('Transfer learning', 'GCN with transfer learning')

In [ ]:
alt.Chart(df[df.Set=='Test']).mark_circle().encode(
    x=alt.X('Train Size:Q', scale=alt.Scale(type='log')),
    y=alt.Y('mse:Q', title='MSE', axis=alt.Axis(format='.1e')),
    color='Model',
    tooltip=['Model', 'mse']
).properties(width=400, height=200, title='Transfer learning - group 9')